<a href="https://colab.research.google.com/github/DeepLearningSaeid/Grad/blob/main/Chapter_3_with_Co_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Libraries

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Install required packages
!pip install autokeras
!pip install keras==2.9.0
!pip install SWAG-DNN==0.1.31

# Import required libraries
from sklearn.model_selection import train_test_split
import autokeras as ak
from SWAG_DNN.taylor import *
import matplotlib.pyplot as plt

In [ ]:

epochs=5
batch_size=1

# define 10-fold cross validation test harness
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
# kfold = KFold(n_splits=10, shuffle=True, random_state=42)

cvscores = []

list_fun=['Ch_First','Ch_second','SWAG','Hermite','Legendre','Sin']

In [ ]:
# This function trains various models using different activation functions and performs
# 10-fold cross-validation to evaluate their performance. The function takes four arguments:
# X (input data), Y (labels), Input_size (number of input features), and output_size
# (number of output features). The activation functions used are stored in the activation_functions list,
# which includes Chebyshev of the First Kind, Chebyshev of the Second Kind, SWAG, Hermite,
# Legendre, and Sin functions.
def train_fun(X, Y, Input_size, output_size):
    activation_functions = ['Ch_First', 'Ch_second', 'SWAG', 'Hermite', 'Legendre', 'Sin']
    results = list()
    
    # Iterate through each activation function
    for act_function in activation_functions:
        print('############################################################################################################################')       
        cv_scores = []
        print(act_function)
        model = globals()[act_function](Input_size, Input_size, output_size)
        print(model.summary())
        
        accuracies = np.zeros(shape=(11,))
        index = 0

        # Perform 10-fold cross-validation
        for train, test in kfold.split(X, Y):
            model.fit(X[train], Y[train], epochs=epochs, batch_size=batch_size, verbose=0)
            scores = model.evaluate(X[test], Y[test], verbose=0)
            print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))
            cv_scores.append(scores[1] * 100)
            accuracies[index] = scores[1] * 100
            index += 1

        # Calculate the mean accuracy and append it to the accuracies array
        accuracies[index] = np.mean(cv_scores)
        results.append(accuracies)
        print("The Mean accuracy is %.2f%% (+/- %.2f%%)" % (np.mean(cv_scores), np.std(cv_scores)))

    # AutoKeras model
    auto_keras_accuracies = np.zeros(shape=(11,))
    index = 0
    cv_scores = []
    print('AutoKeras Model (max_trials=30) Result:')
    
    for train, test in kfold.split(X, Y):
        model = ak.StructuredDataClassifier(max_trials=30)
        model.fit(X[train], Y[train], epochs=epochs, verbose=0)
        scores = model.evaluate(X[test], Y[test], verbose=0)
        print(" accuracy : %.2f%%" % (scores[1] * 100))
        cv_scores.append(scores[1] * 100)
        auto_keras_accuracies[index] = scores[1] * 100
        index += 1
    
    auto_keras_accuracies[index] = np.mean(cv_scores)
    print(np.mean(cv_scores)) 
    results.append(auto_keras_accuracies)
    print('**************************************************************************************************')
    
    return results


In [ ]:
# This function takes the results from the models and prints the accuracy scores for each fold
# and the mean accuracy for each model. The input is a list of arrays (model_re) containing
# accuracy scores for each model.
def print_model_results(model_results):
    # Iterate through each fold (11 in total: 10 for each fold, and 1 for the mean)
    for j in range(11):
        result_str = '0'
        
        # Iterate through each model (7 models in total)
        for i in range(7):
            # Format the result string with accuracy scores
            if i > 0:
                result_str = result_str + ' & ' + str(round(model_results[i][j], 2))
            else:
                result_str = str(round(model_results[i][j], 2))
        
        # Add the LaTeX formatting for a new row in a table
        result_str = result_str + r" \\ "
        
        # Print the results
        if j < 10:
            print(result_str)
        else:
            print('************************************************')
            print(result_str)
            print('************************************************************************************************')
    
    # Print the mean accuracy for each model with its respective name
    model_names = ['Chebyshev First kind', 'Chebyshev Second kind', 'SWAG', 'Hermite', 'Legendre', 'Sin(nX)', "AutoKeras"]
    for i in range(7):
        print(model_names[i], model_results[i])


In [ ]:
# This function trains and evaluates an AutoKeras model using 10-fold cross-validation.
# It takes two input arguments: X_ (features) and Y_ (labels) and returns an array
# containing the accuracy scores for each fold and the mean accuracy.
def auto_keras_model(X_, Y_):
    # Initialize an array to store accuracy scores
    accuracy_scores = np.zeros(shape=(10,))
    index = 0
    cvscores = []

    # Perform 10-fold cross-validation
    for train, test in kfold.split(X_, Y_):
        # Create and train the AutoKeras model
        model = ak.StructuredDataClassifier(max_trials=30)
        model.fit(X_[train], Y_[train], epochs=epochs, verbose=0)
        
        # Evaluate the model and print the accuracy
        scores = model.evaluate(X_[test], Y_[test], verbose=0)
        print(" accuracy : %.2f%%" % (scores[1] * 100))
        
        # Store the accuracy scores
        cvscores.append(scores[1] * 100)
        accuracy_scores[index] = scores[1] * 100
        index += 1

    # Print the mean accuracy
    print(np.mean(cvscores))
    print('**************************************************************************************************')
    
    # Return the accuracy scores array
    return accuracy_scores


#  ========================================================
# First experiment
##  Load data of   ionosphere data set


This is a small dataset that you can <a href="https://www.kaggle.com/creepyghost/uci-ionosphere/version/1">download from the kaggle </a>.


### About this file:

The Ionosphere dataset is a widely-used benchmark dataset in machine learning and pattern recognition. It was originally collected by researchers at the Johns Hopkins University Applied Physics Laboratory for the purpose of studying the ionosphere's structure and electron content. The dataset consists of radar data collected from a phased array of 16 high-frequency antennas, with a total of 34 attributes and a binary target variable.

The dataset contains 351 instances, each representing a radar return from the ionosphere. The 34 attributes correspond to various features extracted from the radar signals, such as the phase and amplitude of the received signal. These features have been pre-processed and scaled to make them suitable for machine learning algorithms. The binary target variable indicates whether the radar return shows evidence of some structure in the ionosphere (labeled "good") or not (labeled "bad").

Source: Donated to UCI Machine Learning Repository by

Donor:

Vince Sigillito (vgs '@' aplcen.apl.jhu.edu)

Source:

Space Physics Group Applied Physics Laboratory Johns Hopkins University Johns Hopkins Road Laurel, MD 20723

In [ ]:
# This code segment loads the ionosphere dataset from the provided URL and stores it as a Pandas DataFrame. It then converts the DataFrame into a NumPy array and separates the features (X) and labels (Y). The class values are encoded as integers, and the models are trained and evaluated using the train_fun function.

# Load the ionosphere dataset from the provided URL and store it as a Pandas DataFrame
dataframe = read_csv('https://raw.githubusercontent.com/DeepLearningSaeid/New-Type-of-Deep-Learning/master/ionosphere_data_kaggle.csv', header=1)

# Convert the DataFrame into a NumPy array
dataset = dataframe.values

# Get the number of rows and columns
num_rows, num_columns = dataset.shape

# Separate the features (X) and labels (Y)
X = dataset[:, 0:num_columns - 1].astype(float)
Y = dataset[:, num_columns - 1]

# Encode the class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y_encoded = encoder.transform(Y)

# Train and evaluate the models using the train_fun function
ionosphere_results = train_fun(X, Y_encoded, num_columns - 1, 1)


### We use 10 k-fold Cross-Validation

In [ ]:
model_result(ionosphere)


#  ========================================================
# Second experiment
##  Load data of  banknote authentication Data Set 

This is a small dataset that you can <a href="https://www.kaggle.com/jacksonharper/data_banknote_authentication">download from the kaggle </a>.


### About this file:

https://www.kaggle.com/jacksonharper/data_banknote_authentication


In [ ]:
dataframe = read_csv('https://raw.githubusercontent.com/DeepLearningSaeid/New-Type-of-Deep-Learning/master/data_banknote_authentication.csv' , header=None)
dataset = dataframe.values


#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
banknote=train_fun(X,Y,Input_size-1,1)


In [ ]:
model_result(banknote)


#  ========================================================
# Third experiment
##  Load data of  Connectionist Bench (Sonar)

This is a small dataset that you can <a href="https://www.kaggle.com/adx891/sonar-data-set">download from the kaggle </a>.


### About this file:

Connectionist Bench (Sonar, Mines vs. Rocks) Data Set Download: Data Folder, Data Set Description


In [ ]:
dataframe = read_csv('https://raw.githubusercontent.com/DeepLearningSaeid/New-Type-of-Deep-Learning/master/sonar.all-data.csv', header=None)
dataset = dataframe.values


#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
sonar=train_fun(X,Y,Input_size-1,1)


In [ ]:
model_result(sonar)


#  ========================================================
# Fourth  experiment    
##  Load data of  Pima Indians Diabetes Database



This is a small dataset that you can <a href="https://www.kaggle.com/uciml/pima-indians-diabetes-database">download from the kaggle </a>.


### About this file:

The datasets consist of several medical predictor (independent) variables and one target (dependent) variable, Outcome. Independent variables include the number of pregnancies the patient has had, their BMI, insulin level, age, and so on.

In [ ]:
dataframe = read_csv('https://raw.githubusercontent.com/DeepLearningSaeid/New-Type-of-Deep-Learning/master/diabetes.csv', header=1)
dataset = dataframe.values


#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
diabetes=train_fun(X,Y,Input_size-1,1)


In [ ]:
model_result(diabetes)


#  ========================================================
# Second experiment
##  Load data of   Iris Data Set


This is a small dataset that you can <a href="https://www.kaggle.com/uciml/iris#Iris.csv">download from the kaggle </a>.


### About this file:

https://www.kaggle.com/uciml/iris#Iris.csv


In [ ]:
dataframe = read_csv('https://raw.githubusercontent.com/DeepLearningSaeid/New-Type-of-Deep-Learning/master/Iris.csv', header=1)
dataset = dataframe.values


# Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.fit_transform(Y)
Y = pd.get_dummies(Y).values

iris=train_fun(X,Y,Input_size-1,3)
# print(Y)

In [ ]:
print(Y)

In [ ]:
from sklearn.utils.multiclass import type_of_target
type_of_target(Y)

In [ ]:
iris=train_fun(X,Y,Input_size-1,3)

#  ========================================================
# Sixth  experiment        
##  Load data of  Red Wine Quality


This is a small dataset that you can download 
<a href="https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009#winequality-red.csv">
Here </a>.


### About this file:

This datasets is related to red variants of the Portuguese "Vinho Verde" wine. For more details, consult the reference [Cortez et al., 2009]. Due to privacy and logistic issues, only physicochemical (inputs) and sensory (the output) variables are available (e.g. there is no data about grape types, wine brand, wine selling price, etc.).

In [ ]:
dataframe = read_csv('https://raw.githubusercontent.com/DeepLearningSaeid/New-Type-of-Deep-Learning/master/winequality-red.csv', header=1)
dataset = dataframe.values

#Number of Rows and columns
Number_rows,Input_size=dataset.shape

X = dataset[:,0:Input_size-1].astype(float)
Y = dataset[:,Input_size-1]
X=normalize(X)
X=rescale_range(X)

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
Y = encoder.transform(Y)
wine=train_fun(X,Y,Input_size-1)

In [ ]:
fig, axs = plt.subplots(2, 2)
fig.set_size_inches(18.5, 10.5)
axs[0, 0].boxplot(ionosphere, labels=[str(r) for r in list_fun], showmeans=True)
axs[0, 0].set_title('ionosphere')
axs[1, 0].boxplot(banknote, labels=[str(r) for r in list_fun], showmeans=True)
axs[1, 0].set_title('banknote')
axs[1, 1].boxplot(sonar, labels=[str(r) for r in list_fun], showmeans=True)
axs[1, 1].set_title('sonar')
axs[0, 1].boxplot(diabetes, labels=[str(r) for r in list_fun], showmeans=True)
axs[0, 1].set_title('diabetes')


In [ ]:
fig, axs = plt.subplots(2, 2)
fig.set_size_inches(20, 20)
axs[0, 0].boxplot(ionosphere, labels=[str(r) for r in list_fun], showmeans=True)
axs[0, 0].set_title('ionosphere')
axs[1, 0].boxplot(banknote, labels=[str(r) for r in list_fun], showmeans=True)
axs[1, 0].set_title('banknote')
axs[1, 1].boxplot(sonar, labels=[str(r) for r in list_fun], showmeans=True)
axs[1, 1].set_title('sonar')
axs[0, 1].boxplot(diabetes, labels=[str(r) for r in list_fun], showmeans=True)
axs[0, 1].set_title('diabetes')


In [ ]:
histories=[]


# Fifth experiment
## Load data of THE MNIST DATABASE of handwritten digits

## About this file:
The MNIST database of handwritten digits, has a training set of 60,000 examples, and a test set of 10,000 examples. It is a subset of a larger set available from NIST. The digits have been size-normalized and centered in a fixed-size image.

In [ ]:
def plot_(histories):    
    plt.rcParams.update({'font.size': 16})
    size=1
    in_1=histories[0]
    in_2=histories[1]
    in_3=histories[2]
    in_4=histories[3]
    in_5=histories[4]
    in_6=histories[5]
    plt.figure(figsize=(15,15))
    #['Ch_First','Ch_second','SWAG','Hermite','Legendre','Sin']
###############################################
    plt.subplot(321)
    training_loss1 = in_1.history['accuracy']
    test_loss1 = in_1.history['val_accuracy']
    epoch_count = range(1, len(training_loss1) + 1)
    plt.plot(epoch_count, training_loss1, 'r--')
    plt.plot(epoch_count, test_loss1, 'b-')
    plt.legend(['Training Loss', 'Test Loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    #plt.ylim(0, size)
    plt.title('Ch_First')

##################################################
    plt.subplot(322)
    training_loss1 = in_2.history['accuracy']
    test_loss1 = in_2.history['val_accuracy']
    epoch_count = range(1, len(training_loss1) + 1)
    plt.plot(epoch_count, training_loss1, 'r--')
    plt.plot(epoch_count, test_loss1, 'b-')
    plt.legend(['Training Loss', 'Test Loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    #plt.ylim(0, size)
    plt.title('Ch_second')
##################################################
    plt.subplot(323)
    training_loss1 = in_3.history['accuracy']
    test_loss1 = in_3.history['val_accuracy']
    epoch_count = range(1, len(training_loss1) + 1)
    plt.plot(epoch_count, training_loss1, 'r--')
    plt.plot(epoch_count, test_loss1, 'b-')
    plt.legend(['Training Loss', 'Test Loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    #plt.ylim(0, size)
    plt.title('SWAG')    
##################################################
    plt.subplot(324)
    training_loss1 = in_4.history['accuracy']
    test_loss1 = in_4.history['val_accuracy']
    epoch_count = range(1, len(training_loss1) + 1)
    plt.plot(epoch_count, training_loss1, 'r--')
    plt.plot(epoch_count, test_loss1, 'b-')
    plt.legend(['Training Loss', 'Test Loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    #plt.ylim(0, size)
    plt.title('Hermite') 
##################################################
    plt.subplot(325)
    training_loss1 = in_5.history['accuracy']
    test_loss1 = in_5.history['val_accuracy']
    epoch_count = range(1, len(training_loss1) + 1)
    plt.plot(epoch_count, training_loss1, 'r--')
    plt.plot(epoch_count, test_loss1, 'b-')
    plt.legend(['Training Loss', 'Test Loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    #plt.ylim(0, size)
    plt.title('Legendre') 
##################################################
    plt.subplot(326)
    training_loss1 = in_6.history['accuracy']
    test_loss1 = in_6.history['val_accuracy']
    epoch_count = range(1, len(training_loss1) + 1)
    plt.plot(epoch_count, training_loss1, 'r--')
    plt.plot(epoch_count, test_loss1, 'b-')
    plt.legend(['Training Loss', 'Test Loss'])
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    #plt.ylim(0, size)
    plt.title('Sin(mx)') 
    
    plt.subplots_adjust(top=0.92, bottom=0.08, left=0.0010, right=0.95, hspace=0.3,
            wspace=0.18)

    plt.show()

In [ ]:
df_XY_train = pd.read_csv('https://raw.githubusercontent.com/sbussmann/kaggle-mnist/master/Data/train.csv')
df_X_test  = pd.read_csv('https://raw.githubusercontent.com/sbussmann/kaggle-mnist/master/Data/test.csv')

Y_train = df_XY_train['label'].values
X_train = df_XY_train.drop('label', axis=1).values
X_test  = df_X_test.values

# Normalize the data

# Normalize the data
X_train = X_train +1
X_test = X_test +1
X_train = X_train / 3.0
X_test = X_test / 3.0

img_rows, img_cols = 28, 28
input_shape = (784, 1) #tensorflow channels_last
num_classes = 10

X_train = X_train.reshape(X_train.shape[0],img_rows*img_cols).astype('float32')/255
Y_train = to_categorical(Y_train, num_classes)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size = 0.1, random_state=7)

In [ ]:
# create model
epochs=3
batch_size=128

for i in list_fun:
    model_7=locals()[i](784,784,10)
    model_7.summary()
    history=model_7.fit(X_train, Y_train, batch_size = batch_size, epochs = epochs,validation_data = (X_val, Y_val))
    histories.append(history)

In [ ]:
plot_(histories)